<a href="https://colab.research.google.com/github/lanpigwen/1st/blob/main/CIFAR10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import torch.nn
import torchvision.datasets
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm
from torch import nn
from torch.nn import Sequential, Conv2d, MaxPool2d, Flatten, Linear

class CifarNN10(nn.Module):
    def __init__(self):
        super().__init__()
        self.module1 = Sequential(
            Conv2d(3, 32, 5, padding=2),
            MaxPool2d(2),
            Conv2d(32, 32, 5, padding=2),
            MaxPool2d(2),
            Conv2d(32, 64, 5, padding=2),
            MaxPool2d(2),
            Flatten(),
            Linear(1024, 64),
            Linear(64, 10)
        )

    def forward(self, x):
        output = self.module1(x)
        return output



# 准备数据
dataset_train = torchvision.datasets.CIFAR10("datasets", train=True, transform=torchvision.transforms.ToTensor(),
                                             download=True)
dataset_test = torchvision.datasets.CIFAR10("datasets", train=False, transform=torchvision.transforms.ToTensor(),
                                            download=True)
# dataloader_train = DataLoader(dataset_train, batch_size=128, drop_last=True)
# dataloader_test = DataLoader(dataset_test, batch_size=128, drop_last=True)

dataloader_train = DataLoader(dataset_train, batch_size=128, drop_last=False)
dataloader_test = DataLoader(dataset_test, batch_size=128, drop_last=False)

train_size = len(dataset_train)
test_size = len(dataset_test)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# 构建模型 从p29model中import
nnModel = CifarNN10()
nnModel.to(device)

# 定义损失函数
loss_fn = torch.nn.CrossEntropyLoss()
loss_fn.to(device)

# 定义优化器
learn_rate = 1e-2
optimizer = torch.optim.SGD(nnModel.parameters(), lr=learn_rate)

# tensorboard
writer = SummaryWriter("p29Log")

total_train_step = 0
total_test_step = 0
num_epoch = 10

for epoch in range(num_epoch):
    # print(f'第{epoch+1}轮训练开始')
    # total_train_loss=0
    total_train_data = 0
    total_train_accuracy = 0

    nnModel.train()
    # nnModel.train()/.eval()只会影响dropout 还有一个什么

    train_loop = tqdm((dataloader_train), total=len(dataloader_train))
    for data in train_loop:
        imgs, targets = data
        imgs, targets = imgs.to(device), targets.to(device)
        outputs = nnModel(imgs)
        loss = loss_fn(outputs, targets)
        # 优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # tensorboard显示
        total_train_step += 1
        writer.add_scalar("train_loss", loss.item(), total_train_step)

        accuracy = (outputs.argmax(1) == targets).sum()
        total_train_accuracy += accuracy
        total_train_data += len(imgs)
        accuracy = total_train_accuracy / float(total_train_data)

        train_loop.set_description(f'Epoch[{epoch + 1}/{num_epoch}] train')
        train_loop.set_postfix(loss=loss.item(), acc=accuracy.item())

        # if total_train_step % 100 == 0:
        #     print(f'----第{total_train_step}次数据训练---- loss:{loss.item()}')

    # 测试时，不要影响grad
    nnModel.eval()
    total_test_loss = 0
    total_test_accuracy = 0
    total_test_data = 0
    with torch.no_grad():
        test_loop = tqdm((dataloader_test), total=len(dataloader_test))
        for data in test_loop:
            imgs, targets = data
            imgs, targets = imgs.to(device), targets.to(device)
            outputs = nnModel(imgs)
            loss = loss_fn(outputs, targets)
            total_test_loss += loss.item()

            accuracy = (outputs.argmax(1) == targets).sum()
            total_test_accuracy += accuracy
            total_test_data += len(imgs)
            accuracy = total_test_accuracy / float(total_test_data)

            test_loop.set_description(f'Epoch[{epoch + 1}/{num_epoch}] test')
            test_loop.set_postfix(loss=loss.item(), acc=accuracy.item())

    total_test_accuracy = total_test_accuracy / float(test_size)
    # test_loop.write(f'第{epoch + 1}轮训练:  loss:{total_test_loss}   accuracy:{total_test_accuracy}')
    # print(f'第{epoch+1}轮训练:  loss:{total_test_loss}   accuracy:{total_test_accuracy}')
    # tensorboard
    total_test_step += 1
    writer.add_scalar("test_loss", total_test_loss, total_test_step)
    writer.add_scalar("test_accuracy", total_test_accuracy, total_test_step)

writer.close()


Files already downloaded and verified
Files already downloaded and verified
cuda


Epoch[10/10] test: 100%|██████████| 79/79 [00:01<00:00, 42.31it/s, acc=0.501, loss=1.44]
